In [3]:
#import the packages, I used re package to remove punctuations, math package to calculate square root and os package to find the file
import numpy as np
import pandas as pd
import requests
import os
import re
import math

In [9]:
#read data
os.chdir(r'/Users/ruyiyang/Desktop/coding_discussions_ppol564_fall2020/04_coding_discussion/Data')
alj = open("aljazeera-khashoggi.txt", 'r', encoding = "UTF-8").read().replace("\n", " ")
bbc = open('bbc-khashoggi.txt', 'r', encoding = "UTF-8").read().replace("\n", " ")
bre = open('breitbart-khashoggi.txt', 'r', encoding = "UTF-8").read().replace("\n", " ")
cnn = open('cnn-khashoggi.txt', 'r', encoding = "UTF-8").read().replace("\n", " ")
fox = open('fox-khashoggi.txt', 'r', encoding = "UTF-8").read().replace("\n", " ")
stop_word_table = pd.read_csv('stop_words.csv')

In [10]:
#make the stop word to list 
stop_word =stop_word_table["word"].to_list()


In [11]:

def tokenize(text=None):
    '''
    remove the words which are in stop_words and remove punctuations from the tokens
    convert word in str to tokens
    return a dictionary indicating the occurance of each tokens
    '''
    d = dict()
    text = text.lower()
    text = re.sub(r"[^\w\s]+", '', text) 
    text_list = text.split()
    text_list = list(filter(lambda a: a not in stop_word, text_list))
    for word in text_list:
        if word in d:
            d[word][0] += 1
        else:
            d[word] = [1]

    return d



In [12]:
def dftable(texts = None):
    '''
    return a dataframe of tokenized values from all news files
    '''
    DTM = pd.DataFrame()
    for text in texts:
        entry = tokenize(text)
        DTM = DTM.append(pd.DataFrame(entry),ignore_index=True,sort=True) # Row bind
    
    DTM.fillna(0, inplace=True)
    return DTM
#generate the table of tokenized value from the 5 news sources       
final = dftable([alj,bbc,bre,cnn,fox]) 
#rename the table for easier understand
final = final.rename({0: "aljazeera", 1: "bbc",2:"breitbart",3:"cnn",4:"fox"}, axis='index')

In [13]:
def cos(a = None, b = None):
    '''
    follows the mathematical way of calculatiing the angle between vectors
    return the cosine value between two vectors
    '''
    sum1 = 0
    sum2 = 0
    sum3 = 0
    
    for i in range(len(a)):
        sum1 += a[i]*b[i]
        sum2 += (a[i])**2
        sum3 += (b[i])**2
    
    return sum1/(math.sqrt(sum2)*math.sqrt(sum3))    


In [14]:
#generate a list of values in the tokenized table
nlist = final.values.tolist()
fillva = []
#run a loop to calculate the cosine value between vectors
for i in nlist:
    for j in nlist:
        fillva.append(cos(i,j))
#break the list to nested lists
fillva = [fillva[x:x+5] for x in range(0, len(fillva), 5)]


In [15]:
#generate the cosine table and rename the index and columns of the table for better understand
costable = pd.DataFrame(fillva, index=["aljazeera", "bbc", "breitbart","cnn","fox"], columns=["aljazeera", "bbc", "breitbart","cnn","fox"])

In [16]:
costable

,aljazeera,bbc,breitbart,cnn,fox
aljazeera,1.000000,0.678938,0.598039,0.533123,0.695217
bbc,0.678938,1.000000,0.596260,0.503919,0.645459
breitbart,0.598039,0.596260,1.000000,0.377132,0.552834
cnn,0.533123,0.503919,0.377132,1.000000,0.534126
fox,0.695217,0.645459,0.552834,0.534126,1.000000


# Discussion
From the table we can find that these news sites vectors generally has a cosine value more than 0.5, which indicates that these news sites report the news in a similar way. Among these data, Alijazeera and Fox and the highest cosine value which is 0.695 and Alijazeera generally has the highest coosine value with other US news sites, which might because the middle-east news publishers are influenced by the United States news sites. CNN has on average the most dissimilar report compared with others. My report would be changed if I remove the words of high frequence such as "Khashoggi" and "Erdogan".